In [11]:
import glob
import os
import pandas as pd
import numpy as np
from pathlib import Path
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.postprocessing as post
from nwb4fp.postprocess.Get_positions import load_positions,load_positions_h5
from nwb4fp.postprocess.get_potential_merge import get_potential_merge
from spikeinterface.preprocessing import (bandpass_filter,
                                           common_reference)
import spikeinterface.exporters as sex
import spikeinterface.qualitymetrics as sqm
from pathlib import Path
import pandas as pd
import probeinterface as pi

In [14]:
vedio_search_directory = r"S:/Sachuriga/Ephys_Vedio/CR_CA1"
path=r"S:/Sachuriga\Ephys_Recording/CR_CA1/65283/65283_2023-10-01_15-14-45_A_phy_k"
raw_path=r"S:\Sachuriga\Ephys_Recording\CR_CA1\65283\65283_2023-10-01_15-14-45_A"
temp_folder=r"C:/temp_wave"
if path.endswith("phy_k_manual"):
    num2cal = int(41)
elif path.endswith("phy_k"):
    num2cal = int(35)

temp = path[0 - num2cal:]
path1 = temp.split("/")
UD = path1[1].split("_")

In [4]:
new_data = pd.DataFrame(columns=['File', 'competability','dlc'])

In [8]:
sorting = se.read_phy(folder_path=path, load_all_cluster_properties=True,exclude_cluster_groups = ["noise", "mua"])
global_job_kwargs = dict(n_jobs=12, chunk_size=10000, chunk_duration="1s", total_memory="32G")
si.set_global_job_kwargs(**global_job_kwargs)
temp_path = path.split("_phy")
raw_path = temp_path[0]
stream_name = 'Record Node 101#OE_FPGA_Acquisition_Board-100.Rhythm Data'
try:
    recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)
except AssertionError:
    try:
        stream_name = 'Record Node 102#OE_FPGA_Acquisition_Board-101.Rhythm Data'
        recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)
    except AssertionError:
        stream_name = 'Record Node 101#Acquisition_Board-100.Rhythm Data'
        recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)


In [19]:
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-F'
probe = pi.get_probe(manufacturer, probe_name)
print(probe)
# probe.wiring_to_device('cambridgeneurotech_mini-amp-64')
# map channels to device indices
mapping_to_device = [
    # connector J2 TOP
    41, 39, 38, 37, 35, 34, 33, 32, 29, 30, 28, 26, 25, 24, 22, 20,
    46, 45, 44, 43, 42, 40, 36, 31, 27, 23, 21, 18, 19, 17, 16, 14,
    # connector J1 BOTTOM
    55, 53, 54, 52, 51, 50, 49, 48, 47, 15, 13, 12, 11, 9, 10, 8,
    63, 62, 61, 60, 59, 58, 57, 56, 7, 6, 5, 4, 3, 2, 1, 0
]

probe.set_device_channel_indices(mapping_to_device)
probe.to_dataframe(complete=True).loc[:, ["contact_ids", "shank_ids", "device_channel_indices"]]
probegroup = pi.ProbeGroup()
probegroup.add_probe(probe)

pi.write_prb(f"{probe_name}.prb", probegroup, group_mode="by_shank")
recording_prb = recording.set_probe(probe, group_mode="by_shank")
rec = bandpass_filter(recording_prb, freq_min=300, freq_max=6000)
rec_save = common_reference(rec, reference='global', operator='median')
sorting.set_property(key='group', values = sorting.get_property("channel_group"))
print(f"Checking the sorting properties")

ASSY-236-F - cambridgeneurotech - 64ch - 6shanks
Checking the sorting properties


In [26]:
try:
    wf = si.extract_waveforms(rec_save, sorting, folder=fr"{temp_folder}", overwrite=True, 
                              sparse=True, method="by_property",by_property="group",max_spikes_per_unit=1000)
    try:
        arr_with_new_col = load_positions_h5(path,vedio_search_directory,raw_path,UD)
        new_row = pd.DataFrame({'File': [raw_path], 'competability': "can be merged",'dlc': "file exits"})
    except IndexError:
        new_row = pd.DataFrame({'File': [raw_path], 'competability': "can be merged",'dlc': "file not found"})
    print(f"{raw_path} merge complete")
except AssertionError:
        try:
            arr_with_new_col = load_positions_h5(path,vedio_search_directory,raw_path,UD)
            new_row = pd.DataFrame({'File': [raw_path], 'competability': "can not be merged",'dlc': "file exits"})
        except IndexError:
            new_row = pd.DataFrame({'File': [raw_path], 'competability': "can not be merged",'dlc': "file not found"})
        print(f"{raw_path} no merge")

extract waveforms memmap multi buffer: 100%|██████████████████████████████████████| 1001/1001 [00:06<00:00, 158.03it/s]


S:/Sachuriga/Ephys_Vedio/CR_CA1\*65283*A2023-10-01*800000_sk_filtered.h5
[]
S:\Sachuriga\Ephys_Recording\CR_CA1\65283\65283_2023-10-01_15-14-45_A merge complete


In [24]:
def load_positions_h5(path,vedio_search_directory,folder_path,UD):
    import glob
    import os
    import pandas as pd
    import numpy as np
    from pathlib import Path

    ''' # Parameters:

        path: This parameter is not used in the function and can be removed.
        vedio_search_directory: This is the directory where the function will look for CSV files containing position data.
        folder_path: This is the directory where the function will look for .npy files containing timestamps and states.
        UD: This is a list or array-like object containing parameters used to construct the search pattern for the CSV files.
        Usage:

        Call the function with the appropriate parameters. For example:
        The function will print the search pattern it uses to find the CSV files.
        The function returns a numpy array containing the position data with timestamps inserted as the first column.
        Output:

        The function returns a numpy array where each row corresponds to a position sample. The first column of the array contains the timestamps, and the remaining columns contain the position data.
        Notes:

        The function assumes a specific directory structure and file naming convention. Make sure your files and directories match these expectations.
        The function only uses the first unique CSV file and the first unique .npy files it finds. If there are multiple matching files, only the first one is used.
        The function extracts specific columns from the CSV file. If your CSV file has a different structure, you may need to modify the column names in the code.
        The function assumes that state 3 in the states.npy file corresponds to the desired timestamps. If your states represent something different, you may need to modify the code.'''
    if path.endswith("phy_k_manual"):
        num2cal = int(41)
    elif path.endswith("phy_k"):
        num2cal = int(35)

    temp = path[0 - num2cal:]
    path1 = temp.split("/")
    UD = path1[1].split("_")

    search_pattern = os.path.join(vedio_search_directory, f"*{UD[0]}*{UD[3]}{UD[1]}*800000_sk_filtered.h5")
    print(search_pattern)
    search_pattern1 = os.path.join(folder_path, '**/**/TTL/timestamps.npy')
    search_pattern3 = os.path.join(folder_path, '**/**/TTL/states.npy')
    #search_pattern2 = os.path.join(folder_path, '**/**/continuous/*/timestamps.npy')

    # Use glob to find files matching the pattern

    matching_files = glob.glob(search_pattern,recursive=True)
    matching_files = np.unique(matching_files)
    print(matching_files)
    try:
        dlc_path=Path(matching_files[0])
        print("Used a 800000 iteration files")
    except IndexError:
        try:
            search_pattern = os.path.join(vedio_search_directory, f"*{UD[0]}*{UD[3]}{UD[1]}*600000_sk_filtered.h5")
            matching_files = glob.glob(search_pattern,recursive=True)
            matching_files = np.unique(matching_files)
            mt_path=matching_files[0]
            dlc_path=Path(mt_path)
            print("Used a 600000 iteration files")
        except IndexError:
            raise IndexError('No file found')
        
    try:
        df = pd.read_hdf(dlc_path, "df_with_missing")
    except KeyError:
        df = pd.read_hdf(dlc_path)   
    bodyparts = df.columns.get_level_values("bodyparts").unique().to_list()
    scorer = df.columns.get_level_values(0)[0]
    
    coords = df[scorer, 'individual1'][[('snout', 'x'), ('snout', 'y'), ('neck', 'x'), ('neck', 'y'), ('back4', 'x'), ('back4', 'y')]]
    positions = np.float32(coords.to_numpy())
    
    print(positions.shape)
    matching_files = glob.glob(search_pattern1,recursive=True)
    matching_files = np.unique(matching_files)
    print(matching_files)
    v_time = np.load(matching_files[0])
    matching_files = glob.glob(search_pattern3,recursive=True)
    matching_files = np.unique(matching_files)
    v_state = np.load(matching_files[0])
    f_time = v_time[np.where(v_state==3)[0]]
    if f_time.shape[0] == 0:
        try:
            f_time = v_time[np.where(v_state==6)[0]]
        except ValueError:
            f_time = v_time[np.where(v_state==3)[0]]
        print('Vedio is 25Hz')

    arr_with_new_col =  np.insert(positions , 0, f_time[:len(positions)], axis=1) # type: ignore
    return arr_with_new_col